In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd
import sqlite3

In [22]:
# Conectar ao banco de dados
db_path = '/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DADOS PROJETO BI/Experimentai.db'
conn = sqlite3.connect(db_path)

# Carregar os dados das tabelas Assinaturas e Pagamentos
assinaturas_df = pd.read_sql_query("SELECT * FROM Assinaturas", conn)
pagamentos_df = pd.read_sql_query("SELECT * FROM Pagamentos", conn)
assinaturasDemo = pd.read_sql_query("SELECT * FROM DimDemograficoBaseNova", conn)
cliente_fato_df = pd.read_sql_query("SELECT renda_id, id_fato, id_cliente FROM clienteFato", conn)


In [23]:
cliente_fato_df.head()

,renda_id,id_fato,id_cliente
0,R$16.000 - 24.000,1,17
1,R$0 - 3.000,2,18
2,R$3.000 - 8.000,3,20
3,R$8.000 - 16.000,4,22
4,R$3.000 - 8.000,5,23


In [8]:
assinaturas_df.loc[assinaturas_df['Status'] == 'active', 'Churn'] = 0
assinaturas_df.loc[assinaturas_df['Status'] != 'active', 'Churn'] = 1

In [ ]:
assinaturasDemo = assinaturasDemo[['idade', 'renda_id', 'estado',]]

In [9]:
pagamentos_df['Amount Refunded'] = pd.to_numeric(pagamentos_df['Amount Refunded'], errors='coerce')
pagamentos_df.loc[pagamentos_df['Amount Refunded'].isna(), 'Amount Refunded'] = 0

In [10]:
pagamentos_summary = pagamentos_df.groupby('Customer ID').agg({
    'Status': lambda x: (x == 'Failed').sum(),
    'Amount Refunded': 'sum'
}).rename(columns={'Status': 'Failed Payments', 'Amount Refunded': 'Total Refunded'})


In [11]:
assinaturas_churn_df = assinaturas_df.merge(pagamentos_summary, on='Customer ID', how='left')
assinaturas_churn_df.loc[assinaturas_churn_df['Failed Payments'].isna(), 'Failed Payments'] = 0
assinaturas_churn_df.loc[assinaturas_churn_df['Total Refunded'].isna(), 'Total Refunded'] = 0

In [12]:
churn_data = assinaturas_churn_df[['Amount', 'Interval', 'Failed Payments', 'Total Refunded', 'Churn']]
churn_data = pd.get_dummies(churn_data, columns=['Interval'], drop_first=True)

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

X = churn_data.drop(columns=['Churn'])
y = churn_data['Churn']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Treinamento e Avaliação
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)

print(classification_report(y_pred, y_test))

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Resultado
accuracy, conf_matrix


              precision    recall  f1-score   support

         0.0       0.96      0.89      0.93       360
         1.0       0.22      0.48      0.31        23

    accuracy                           0.87       383
   macro avg       0.59      0.69      0.62       383
weighted avg       0.92      0.87      0.89       383



(0.8694516971279374,
 array([[322,  12],
        [ 38,  11]]))

In [15]:
print(class_report)

              precision    recall  f1-score   support

         0.0       0.89      0.96      0.93       334
         1.0       0.48      0.22      0.31        49

    accuracy                           0.87       383
   macro avg       0.69      0.59      0.62       383
weighted avg       0.84      0.87      0.85       383



In [24]:
# Load active and canceled subscriptions data
active_subscriptions = pd.read_csv("/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DADOS PROJETO BI/subscriptions (30).csv")
canceled_subscriptions = pd.read_csv("/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DADOS PROJETO BI/subscriptions (31).csv")

# Ensure date columns are in datetime format
active_subscriptions['Created (UTC)'] = pd.to_datetime(active_subscriptions['Created (UTC)'])
canceled_subscriptions['Created (UTC)'] = pd.to_datetime(canceled_subscriptions['Created (UTC)'])
canceled_subscriptions['Canceled At (UTC)'] = pd.to_datetime(canceled_subscriptions['Canceled At (UTC)'])

# Create DataFrames for subscription events
active_events = active_subscriptions[['Created (UTC)']].copy()
active_events['Event'] = 1  # +1 for each new active subscription

canceled_events = canceled_subscriptions[['Created (UTC)', 'Canceled At (UTC)']].copy()
canceled_events['Event'] = 1  # +1 for each subscription start
canceled_events_canceled = canceled_events[['Canceled At (UTC)', 'Event']].copy()
canceled_events_canceled.columns = ['Created (UTC)', 'Event']  # Renaming for consistency
canceled_events_canceled['Event'] = -1  # -1 for each cancellation

# Concatenate all events and sort by date
all_events = pd.concat([active_events, canceled_events[['Created (UTC)', 'Event']], canceled_events_canceled])
all_events = all_events.sort_values(by='Created (UTC)').reset_index(drop=True)

# Calculate cumulative active subscribers
all_events['Active Subscribers'] = all_events['Event'].cumsum()

# Filter the DataFrame for the desired time period
start_date = '2023-11-01'

filtered_events = all_events[(all_events['Created (UTC)'] >= start_date)]

# Plot the growth of active subscribers over time for the filtered period
plt.figure(figsize=(12, 6))
plt.plot(filtered_events['Created (UTC)'], filtered_events['Active Subscribers'], label='Active Subscribers', color='blue')
plt.title('Growth of Active Subscribers Over Time (Filtered Period)')
plt.xlabel('Date')
plt.ylabel('Number of Active Subscribers')
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'plt' is not defined